Transforms can be applied to PIL images, tensors, ndarrays, or custom data  
during creation of the DataSet  

[complete list of built-in transforms](https://pytorch.org/docs/stable/torchvision/transforms.html)  
[non-dead link](https://pytorch.org/vision/0.9/transforms.html)

On images  
---------
CenterCrop, Grayscale, Pad, RandomAffine  
RandomCrop, RandomHorizontalFlip, RandomRotation  
Resize, Scale

On Tensors
----------
LinearTransformation, Normalize, RandomErasing

Conversion
----------
ToPILImage: from tensor or ndrarray  
ToTensor : from numpy.ndarray or PILImage

Generic
-------
Use Lambda

Custom
------
Write own class

Compose multiple Transforms
---------------------------
composed = transforms.Compose([Rescale(256), RandomCrop(224)])

In [1]:
# https://github.com/patrickloeber/pytorchTutorial/blob/master/10_transformers.py

import numpy as np
import torch
import torchvision
from torch.utils.data import Dataset

In [2]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        xy = np.loadtxt("ds/wine.csv",
                        delimiter=",",
                        dtype=np.float32,
                        skiprows=1)
        self.n_samples=xy.shape[0]
        
        # note that we don't convert to tensor here
        self.x_data = xy[:, 1:]
        self.y_data = xy[:, [0]]
        
        self.transform = transform
        
    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]
        
        # will only execute if you provide a transformjj
        if self.transform:
            sample = self.transform(sample)
        
        return sample
    
    def __len__(self):
        return self.n_samples
    
# custom transforms
# implement __call__(self, sample)
class ToTensor:
    # convert ndarrays to tensors
    # __call__ makes it so that this class can be called as if it were a function
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)
    
class MulTransform:
    # multiply inputs with a given factor
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets

In [3]:
print("Without transform")
dataset = WineDataset() # <class 'numpy.ndarray'>
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)

Without transform
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03] [1.]


In [4]:
print("With tensor transform")
# even though ToTensor requires a sample parameter, it is not needed here
dataset = WineDataset(transform=ToTensor()) # converted to <class 'torch.Tensor'>
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)

With tensor transform
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


In [5]:
print("With tensor and multiplication transform")
# compose multiple transforms together
# the 4 in MulTransform is factor, not sample, as it is called from __init__()
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(4)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)

With tensor and multiplication transform
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
        1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
        4.2600e+03]) tensor([1.])
